In [2]:
from sickle import Sickle
import xml.etree.ElementTree as ET
import pprint

sickle = Sickle('https://texashistory.unt.edu/oai')
#records = sickle.ListRecords(metadataPrefix='oai_dc')

In [73]:
# other metadata formats:
#untl - 2159
# oai_dc - 2157
# untl_raw - 2159
# untl_dpla - 2159
# rdf - 2159

records = sickle.ListRecords(metadataPrefix='oai_dc', set='collection:SJCC')

In [10]:
#records.next()

<Record info:ark/67531/metapth1634318>

In [50]:
# count of records
print(sum(1 for e in records))

2159


In [75]:
rec_count = 0
for record in records:
    #print(record.metadata)
    rec_count += 1
    #print(record)
    #pprint.pprint(record.metadata)
    pprint.pprint(record.header)
    #print(record.xml)
    #pprint.pprint(record.metadata)
    #pprint.pprint(record.metadata['identifier'])

    
    #print(record.metadata['title'])
    #print(record.metadata.get('date'))
    #print(record.metadata['coverage'])
    #print(type(record.xml))
    xml_str = ET.tostring(record.xml, encoding='unicode')
    print(xml_str)
    #root = record.xml.getroot()
    #print(root)
    #xml_tree = ET.parse(str(record.xml))
    #xml_tree = ET.parse(record.xml)
    #print(xml_tree['metadata'])
print(rec_count)

<Header info:ark/67531/metapth1634319>
<class 'lxml.etree._Element'>
<ns0:record xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.openarchives.org/OAI/2.0/" xmlns:ns1="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><ns0:header><ns0:identifier>info:ark/67531/metapth1634319</ns0:identifier><ns0:datestamp>2024-08-26T12:47:59Z</ns0:datestamp><ns0:setSpec>partner:BRIT</ns0:setSpec><ns0:setSpec>collection:SJCC</ns0:setSpec><ns0:setSpec>access_rights:public</ns0:setSpec></ns0:header><ns0:metadata><ns1:dc xsi:schemaLocation="http://purl.org/dc/elements/1.1/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"><dc:title>[Kangaroo laying on the ground in Australia]</dc:title><dc:creator>Carlquist, Sherwin John, 1930-2021</dc:creator><dc:subject>Scientific expeditions</dc:subject><dc:subject>Landscape and Nature - Wildlife</dc:subject><dc:subject>color slides</dc:subject><dc:subject>animals</dc:subject><dc:subject>fieldwork</dc:subje

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



<ns0:record xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.openarchives.org/OAI/2.0/" xmlns:ns1="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><ns0:header><ns0:identifier>info:ark/67531/metapth1673367</ns0:identifier><ns0:datestamp>2024-09-04T00:47:10Z</ns0:datestamp><ns0:setSpec>partner:BRIT</ns0:setSpec><ns0:setSpec>collection:SJCC</ns0:setSpec><ns0:setSpec>access_rights:public</ns0:setSpec></ns0:header><ns0:metadata><ns1:dc xsi:schemaLocation="http://purl.org/dc/elements/1.1/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"><dc:title>[Buildings, Bangkok, Thailand]</dc:title><dc:creator>Carlquist, Sherwin John, 1930-2021</dc:creator><dc:subject>Scientific expeditions</dc:subject><dc:subject>Biogeography</dc:subject><dc:subject>color slides</dc:subject><dc:subject>Architecture - Buildings</dc:subject><dc:subject>People</dc:subject><dc:subject>fieldwork</dc:subject><dc:subject>field expeditions</dc:subject><dc:descr

In [43]:
# one record
one_rec = sickle.GetRecord(identifier='info:ark/67531/metapth1634321', metadataPrefix='oai_dc')

In [44]:
print(one_rec)

<record xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><header><identifier>info:ark/67531/metapth1634321</identifier><datestamp>2024-08-26T12:35:29Z</datestamp><setSpec>partner:BRIT</setSpec><setSpec>collection:SJCC</setSpec><setSpec>access_rights:public</setSpec></header><metadata><oai_dc:dc xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xsi:schemaLocation="http://purl.org/dc/elements/1.1/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"><dc:title>[Emu in Australia]</dc:title><dc:creator>Carlquist, Sherwin John, 1930-2021</dc:creator><dc:subject>Scientific expeditions</dc:subject><dc:subject>Landscape and Nature - Wildlife</dc:subject><dc:subject>color slides</dc:subject><dc:subject>animals</dc:subject><dc:subject>fieldwork</dc:subject><dc:subject>field expeditions</dc:subject><dc:description>Photograph of an emu in Australia. The emu is brown with a dark head and is facing the l